In [3]:
''' This sets up the Django environment '''
import os
import django
PROJECTPATH = ""
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mus.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"  # https://docs.djangoproject.com/en/4.1/topics/async/#async-safety
django.setup()



In [41]:
from PureOpenAlex.models import Paper, PureEntry
from django.db.models import Q
from django.db import transaction
from pprint import pprint
for item in PureEntry.objects.exclude(other_links=dict()).only('id','other_links'):
    more=False
    for key, value in item.other_links.items():
            more=True
            break
    if more:
        pprint(item.other_links)
        if item.duplicate_ids:
            pprint(item.duplicate_ids)
        print(item.id)
        print('========================')
        
'''
for item in PureEntry.objects.exclude(duplicate_ids=dict()).only('other_links')[:100]:
    print()
    print(item.other_links)'''

{'other': ['https://www.usenix.org/conference/usenixsecurity20/presentation/lee-hyeonmin']}
54712
{'itc_content': ['http://www.itc.nl/library/papers_2017/pres/ostermann_cit_ppt.pdf']}
66489
{'itc_content': ['https://library.itc.utwente.nl/login/2020/isi/maghsoudimoud_hyb.pdf']}
{'doi': ['https://doi.org/10.1007/s00366-020-01078-9']}
41077
{'itc_content': ['https://library.itc.utwente.nl/login/2022/isi/hoefsloot_dat.pdf']}
41191
{'zenodo': ['https://zenodo.org/record/7515977#.Y8FQDHbMKUk']}
41585
{'itc_content': ['https://library.itc.utwente.nl/login/2022/ref/mannaerts_est.pdf']}
42225
{'other': ['https://utwente.yuja.com/V/Video?v=512346&node=&a=1597802179&autoPlay=1']}
42750
{'other': ['https://opg.optica.org/viewmedia.cfm?uri=QUANTUM-2022-QW4B.2&seq=0']}
45994
{'other': ['https://www.ruhr-uni-bochum.de/schematicmapping/papers/smw-listabarth-raposo.pdf']}
47464
{'other': ['https://patents.google.com/patent/US20220131342A1/en',
           'http://chrome-extension://efaidnbmnnnibpcajpcg

"\nfor item in PureEntry.objects.exclude(duplicate_ids=dict()).only('other_links')[:100]:\n    print()\n    print(item.other_links)"

In [51]:
PureEntry.objects.get(pk=51688).paper


<Paper: Paper object (45790)>

In [2]:
from PureOpenAlex.models import PureAuthor, Author
from django.db import transaction
from nameparser import HumanName
from PureOpenAlex.namematcher import NameMatcher
from unidecode import unidecode
from pprint import pprint

allpureauthors = PureAuthor.objects.all()
purenames={}
purefullnames = {}
pureinitials = {}
for author in allpureauthors:
    hname=HumanName(unidecode(author.name),initials_format="{first} {middle}")
    purenames[author.id] = {
        'full': hname.full_name,
        'initials': hname.initials()+" "+hname.last
    }
    purefullnames[hname.full_name]=author.id
    pureinitials[hname.initials()+" "+hname.last]=author.id
allauthors = Author.objects.all()

authnames={}
authfullnames = {}
authinitials = {}
for author in allauthors:
    hname=HumanName(unidecode(author.name),initials_format="{first} {middle}")
    authnames[author.id] = {
        'full': hname.full_name,
        'initials': hname.initials()+" "+hname.last
    }
    authfullnames[hname.full_name]=author.id
    authinitials[hname.initials()+" "+hname.last]=author.id


purefullnameset=set(purefullnames.keys())
pureinitialsset=set(pureinitials.keys())

authfullnameset=set(authfullnames.keys())
authinitialsset=set(authinitials.keys())

print('# pure authors in db',allpureauthors.count())
print('# pure authors in set (unique names)',len(purefullnameset))
print('# authors in db',allauthors.count())
print('# authors in set (unique names)',len(authfullnameset))

intersection = purefullnameset.intersection(authfullnameset)
print('# common names',len(intersection))

listtosave=[]
from PureOpenAlex.models import PureEntry

j=0
h=0

for i,name in enumerate(intersection):
    pureauthorid=purefullnames[name]
    authorid=authfullnames[name]
    pureauthor = PureAuthor.objects.get(id=pureauthorid)
    pureentries = pureauthor.pure_entries.all()
    pureentry_c = pureauthor.pure_creators.all()
    author=Author.objects.get(id=authorid)
    for entry in pureentries:
        if author not in entry.authors.all():
            entry.authors.add(author)
            listtosave.append(entry)
            h=h+1
    for entry in pureentry_c:
        if author not in entry.authors.all():
            entry.authors.add(author)
            listtosave.append(entry)
            j=j+1
    if i%1000==0:
        print('# of entries that need updating:', len(listtosave))
        print('# of intersections checked:', i)
        print('pureentries:',h)
        print('purecreators:',j)



# pure authors in db 84733
# pure authors in set (unique names) 83350
# authors in db 71001
# authors in set (unique names) 70875


In [17]:

from PureOpenAlex.models import Author,PureEntry
from django.db.models import Q
noauths=PureEntry.objects.filter(authors__isnull=True).distinct()
print(noauths.count())
print(noauths.filter(Q(creators__isnull=False) | Q(contributors__isnull=False)).distinct().count())

5972
5851


In [5]:
def migrate_department_data():
    from PureOpenAlex.models import UTData, Department
    from collections import defaultdict
    from django.db import transaction

    utdatalist=UTData.objects.all().only("departments").prefetch_related("departments")
    facultylist = ['EEMCS', 'BMS', 'ET', 'ITC', 'TNW']
    savelist=[]
    i=0
    j=0
    for data in utdatalist:
        depts=list(data.departments.all())
        i=i+1
        data.employment_data = defaultdict(list)
        if not depts:
            data.current_faculty=""
            data.current_group=""
            data.employment_data['employment'].append({})
        elif len(depts)==1:
            j=j+1
            data.current_faculty=depts[0].faculty
            data.current_group=depts[0].name
            data.employment_data['employment'].append({'faculty':depts[0].faculty,'group':depts[0].name})
        else:
            current=False
            for dept in depts:
                j=j+1
                if not current:
                    if dept.faculty in facultylist:
                        data.current_faculty=dept.faculty
                        data.current_group=dept.name
                        current=True
                data.employment_data['employment'].append({'faculty':dept.faculty,'group':dept.name})
            if not current:
                data.current_faculty=data.employment_data['employment'][0]['faculty']
                data.current_group=data.employment_data['employment'][0]['group']
        savelist.append(data)
        if i%100==0 or i==len(utdatalist):
            print(f'{i} UTData entries with {j} related Departments processed')
            with transaction.atomic():
                UTData.objects.bulk_update(savelist, ['current_faculty', 'current_group', 'employment_data'])
            savelist=[]


migrate_department_data()

100 UTData entries with 104 related Departments processed
200 UTData entries with 208 related Departments processed
300 UTData entries with 312 related Departments processed
400 UTData entries with 413 related Departments processed
500 UTData entries with 519 related Departments processed
600 UTData entries with 627 related Departments processed
700 UTData entries with 728 related Departments processed
800 UTData entries with 832 related Departments processed
900 UTData entries with 946 related Departments processed
1000 UTData entries with 1054 related Departments processed
1100 UTData entries with 1162 related Departments processed
1200 UTData entries with 1263 related Departments processed
1300 UTData entries with 1364 related Departments processed
1400 UTData entries with 1467 related Departments processed
1500 UTData entries with 1574 related Departments processed
1600 UTData entries with 1680 related Departments processed
1700 UTData entries with 1783 related Departments processe

In [13]:
from django.db import transaction
from PureOpenAlex.models import (
    PureEntry,
    Paper,
)

def matchPureEntryWithPaper():
    """
    For every PureEntry, try to find a matching paper in the database and mark them as such.
    """
    paperlist = []
    entrylist = []
    i=0
    j=0
    allentries = PureEntry.objects.all().only("doi","title",'researchutwente', 'risutwente', 'other_links', 'duplicate_ids')
    paperpreload = Paper.objects.all().only("doi","title",'locations','id').prefetch_related('locations')
    for entry in allentries:
        j=j+1
        found=False
        if entry.paper != None or entry.paper == "":
            found=True
        paper = None
        if entry.doi != "":
            paper = paperpreload.filter(doi=entry.doi).first()
        if not paper and entry.risutwente != "":
            paper = paperpreload.filter(locations__pdf_url__icontains=entry.risutwente).first()
        if not paper and entry.researchutwente != "":
            paper = paperpreload.filter(locations__pdf_url__icontains=entry.researchutwente).first()
        if not paper and entry.risutwente != "":
            paper = paperpreload.filter(locations__landing_page_url__icontains=entry.risutwente).first()
        if not paper and entry.researchutwente != "":
            paper = paperpreload.filter(locations__landing_page_url__icontains=entry.researchutwente).first()
        if not paper:
            paper = paperpreload.filter(title__icontains=entry.title).first()
        if not paper and entry.duplicate_ids!={}:
            for key, value in entry.duplicate_ids.items():
                if not paper:
                    if key == 'doi':
                        paper = paperpreload.filter(doi=value).first()
                    if key == 'risutwente' or key == 'researchutwente':
                        paper = paperpreload.filter(locations__pdf_url__icontains=value).first()
                        paper = paperpreload.filter(locations__landing_page_url__icontains=value).first()
        if not paper and entry.other_links!={}:
            if 'other' in entry.other_links:
                for value in entry.other_links['other']:
                    if not paper:
                        paper = paperpreload.filter(locations__pdf_url__icontains=value).first()
                        paper = paperpreload.filter(locations__landing_page_url__icontains=value).first()

        if paper and not found:
            entry.paper = paper
            paper.has_pure_oai_match = True
            paperlist.append(paper)
            entrylist.append(entry)
        elif paper and found:
            if paper.title != entry.paper.title or paper.doi != entry.paper.doi:
                entry.paper = paper
                paper.has_pure_oai_match = True
                paperlist.append(paper)
                entrylist.append(entry)
        else:
            pass #no match or no new match
        if j == 1000:
            with transaction.atomic():
                Paper.objects.bulk_update(paperlist, fields=["has_pure_oai_match"])
                PureEntry.objects.bulk_update(entrylist, fields=["paper"])
            i=i+len(paperlist)
            print("+",str(len(paperlist)), "total:", str(i))
            j=0
            paperlist = []
            entrylist = []


    return True

matchPureEntryWithPaper()

+ 11 total: 11
+ 13 total: 24
+ 40 total: 64
+ 28 total: 92
+ 34 total: 126
+ 36 total: 162
+ 26 total: 188
+ 21 total: 209
+ 19 total: 228
+ 10 total: 238
+ 16 total: 254
+ 14 total: 268
+ 7 total: 275
+ 7 total: 282
+ 7 total: 289
+ 11 total: 300
+ 3 total: 303
+ 4 total: 307
+ 2 total: 309
+ 4 total: 313
+ 5 total: 318
+ 5 total: 323
+ 8 total: 331
+ 29 total: 360
+ 26 total: 386
+ 14 total: 400
+ 27 total: 427
+ 15 total: 442
+ 16 total: 458
+ 14 total: 472
+ 30 total: 502
+ 39 total: 541


In [7]:
from PureOpenAlex.models import Identifier, PureEntry
from collections import defaultdict
from django.db import transaction

MATCHURLCONTENT = {
    "itc.utwente.nl": "itc_content",
    "www.itc.nl": "itc_content",
    "arxiv": "arxiv",
    "zenodo": "zenodo",
    "github": "github",
    "https://10.": "doi",
    "http://10.": "doi",
}
MATCHIDTYPES = {
"doi": '',
"isbn": '',
"researchutwente": '',
"risutwente": '',
"scopus": ''
}
bulklist=[]
i=0
j=0
allentries=PureEntry.objects.all().filter(identifiers__isnull=False).only('doi', 'isbn', 'researchutwente', 'risutwente', 'scopus', 'other_links','id', 'duplicate_ids').prefetch_related("identifiers")
for entry in allentries:
    entry.doi = ""
    entry.isbn = ""
    entry.researchutwente = ""
    entry.risutwente = ""
    entry.scopus = ""
    entry.other_links= defaultdict(list)
    entry.duplicate_ids = defaultdict(list)
    for identifier in entry.identifiers.all():
        j=j+1
        duplicate=False
        if 'https://ezproxy2.utwente.nl/login?url=' in identifier.url:
            identifier.url = identifier.url.replace('https://ezproxy2.utwente.nl/login?url=','')
        if str(identifier.idtype) in MATCHIDTYPES.keys():
            if str(identifier.idtype) == 'doi':
                identifier.url = identifier.url.replace('doi.org1','doi.org/1')
                if entry.doi == "" or entry.doi == None:
                    entry.doi = identifier.url
                else:
                    duplicate=True
            if str(identifier.idtype) == 'isbn':
                identifier.url = identifier.url.strip('urn:ISBN:')
                if entry.isbn == "" or entry.isbn == None:
                    entry.isbn = identifier.url
                else:
                    duplicate=True
            if identifier.idtype == 'researchutwente':
                if entry.researchutwente == "" or entry.researchutwente == None:
                    entry.researchutwente = identifier.url
                else:
                    duplicate=True
            if identifier.idtype == 'risutwente':
                if entry.risutwente == "" or entry.risutwente == None:
                    entry.risutwente = identifier.url
                else:
                    duplicate=True
            if identifier.idtype == 'scopus':
                if entry.scopus == "" or entry.scopus == None:
                    entry.scopus = identifier.url
                else:
                    duplicate=True
            if duplicate:
                entry.duplicate_ids[str(identifier.idtype)].append(identifier.url)
        else:
            matched=False
            for key, value in MATCHURLCONTENT.items():
                if key in identifier.url and not matched:
                    if value != "doi":
                        entry.other_links[value].append(identifier.url)
                        matched=True
                    else: # doi with wrong formatting found
                        identifier.url = identifier.url.replace('doi.org1','doi.org/1')
                        if 'http://' in str(identifier.url) and not 'doi.org' in str(identifier.url):
                            identifier.url=str(identifier.url).replace('http://', 'https://doi.org/')
                        elif 'https://' in str(identifier.url) and not 'doi.org' in str(identifier.url):
                            identifier.url=str(identifier.url).replace('https://', 'https://doi.org/')
                        else:
                            identifier.url=str(identifier.url)
                        if not entry.doi or entry.doi=="":
                            entry.doi=identifier.url
                        elif identifier.url != entry.doi and identifier.url not in entry.duplicate_ids['doi']:
                            entry.duplicate_ids['doi'].append(identifier.url)
                        matched=True
            if not matched:
                if identifier.idtype=="other":
                    entry.other_links['other'].append(identifier.url)
                else:
                    print("idtype not found/not matched", identifier.idtype, identifier.url)
    bulklist.append(entry)
    if len(bulklist)==1000:
        with transaction.atomic():
            PureEntry.objects.bulk_update(bulklist, ['doi', 'isbn', 'researchutwente', 'risutwente', 'scopus', 'other_links', 'duplicate_ids'])
        bulklist=[]
        i=i+1000
        print(str(i) + " entries done")
        print(str(j) + " identifiers processed in total")


1000 entries done
3472 identifiers processed in total
2000 entries done
6964 identifiers processed in total
3000 entries done
10444 identifiers processed in total
4000 entries done
13984 identifiers processed in total
5000 entries done
17474 identifiers processed in total
6000 entries done
20834 identifiers processed in total
7000 entries done
24367 identifiers processed in total
8000 entries done
28158 identifiers processed in total
9000 entries done
31636 identifiers processed in total
10000 entries done
35075 identifiers processed in total
11000 entries done
38420 identifiers processed in total
12000 entries done
41778 identifiers processed in total
13000 entries done
45344 identifiers processed in total
14000 entries done
49031 identifiers processed in total
15000 entries done
52698 identifiers processed in total
16000 entries done
56292 identifiers processed in total
17000 entries done
59767 identifiers processed in total
18000 entries done
63482 identifiers processed in total
190

In [ ]:
from PureOpenAlex.data_repair import matchAFASwithAuthor
results=matchAFASwithAuthor()

space=""
accepted=[]
rejected=[]
for result in results:
    if result[1]==1.0:
        accepted.append(result)
        continue
    curlen=len(f"{result[2].first} {result[2].last}")
    if curlen > len(space):
        space=" ".join(["" for x in range(curlen)])
    rejected.append(result)

i=0
keep=[2,9,11,13,19]
for result in rejected:
    acceptedcheck=""
    extraspace=""
    extranum=5
    if i<10:
        extraspace= " "
    if i in keep:
        accepted.append(result)
        acceptedcheck="[X]"
        extranum=2

    curspace=" ".join(["" for x in range(extranum+len(space)-len(f"{result[2].first} {result[2].last}"))])


    print(f"[{i}]{acceptedcheck} {result[2].first} {result[2].last}{curspace}{extraspace}[{int(result[1]*100)}]   {result[3]}")
    i+=1




In [ ]:
from PureOpenAlex.models import Author
space=""
for result in accepted:
    curlen=len(f"{result[2].first} {result[2].last}")
    if curlen > len(space):
        space=" ".join(["" for x in range(curlen)])
i=0
accept=[]
other=[]
reject=[]

#dict: first one is the i-index of result, second is 0 (no match), 1 (first match), 2 (second match), 3 (other)
# if there is only 1 match alway accept expect if overruled by the dict below
# if there are more than 2 matches mark as other.

final={5:1, 6:2, 8:3, 15:2, 17:3, 18:3, 21:3, 26:3, 28:3, 29:3, 30:2, 31:2, 34:0, 38:1, 43:3, 47:3, 48:3, 50:0, 57:1, 59:0, 62:3, 63:3, 65:1, 67:0, 68:3, }

for result in accepted:
    print("---------------------")
    matchedauthors=Author.objects.filter(name__icontains=" ".join([result[2].first, result[2].last]))
    if matchedauthors.count()==0:
        matchedauthors=Author.objects.filter(first_name__icontains=result[2].first, last_name__icontains=result[2].last)
        if matchedauthors.count()==0:
            matchedauthors=Author.objects.filter(last_name__icontains=result[2].last)


    curspace=" ".join(["" for x in range(2+len(space)-len(f"{result[2].first} {result[2].last}"))])
    extraspace=""
    if result[1]!=1.0:
        extraspace=" "

    print(f"[{i}] {result[2].first} {result[2].last}{curspace}[{int(result[1]*100)}]{extraspace}   {result[3]}")
    if matchedauthors.count()==2:
        print(f"          2 matches found: {matchedauthors.first().name} and {matchedauthors.last().name}")
        if final[i]==0:
            print("Discarded.")
            reject.append([result,None])
        elif final[i]==1:
            print(f"Accepted {matchedauthors.first().name}.")
            accept.append([result,matchedauthors.first()])
        else:
            print(f"Accepted {matchedauthors.last().name}.")
            accept.append([result,matchedauthors.last()])
    elif matchedauthors.count()>1:
        print(f"          {matchedauthors.count()} matches found.")
        print("To others.")
        other.append([result,matchedauthors])
    elif matchedauthors.count()==0:
        print(f"          No matches found.")
        reject.append([result,None])
    else:
        print(f"          Match: {matchedauthors.first().name}")
        try:
            if final[i]==0:
                print("!DISCARDED!")
                reject.append([result,None])
            elif final[i]==1:
                print(f"Accepted.")
                accept.append([result,matchedauthors.first()])
            elif final[i]==3:
                print(f'To others.')
                other.append([result,matchedauthors.first()])
        except:
            print("Accepted.")
            accept.append([result,matchedauthors.first()])
    i+=1

print(f"Accepted: {len(accept)}, Rejected: {len(reject)}, Other: {len(other)}")



In [ ]:
i=0
matching={0:42, 2:2, 3:28, 4:1, 7:0, 12:10, 15:10, 16:11}
extraaccepted=[]
print(len(accept))
print(len(reject))
for entry in other:
    #print("========================")

    #print(f"[{i}]Name:", entry[0][2])
    #print("Found authors:")
    j=0
    authorindex=None
    try:
        authorindex=matching[i]
    except:
        pass

    if type(entry[1]) is not Author:
        for author in entry[1]:
            #print(f"{i}:{j}",author.name)
            if authorindex is not None:
                if j==authorindex:
                    accept.append([entry,author])
                    extraaccepted.append([entry,author])
            j=j+1
    else:
        #print(f"{i}:{j}",author.name)
        pass
    i=i+1

print(len(accept))


In [ ]:
i=0
for entry in accept:
    print(f"accept[{i}] has a list with details (accept[{i}][0]) for author {entry[1].name} (accept[{i}][1]) ")
    print(f"[{i}][0][0]: openalex api response for author")
    print(f"[{i}][0][1]: matching score")
    print(f"[{i}][0][2]: initial matching name from AFASdata")
    print(f"[{i}][0][3]: matched name in openalex")
    print(entry[0][2]," -- ", entry[0][3])
    i=i+1






In [ ]:
from PureOpenAlex.models import AFASData
from django.db import transaction
for entry in accept:
    try:
        name=entry[0][2].full_name
    except:
        name = entry[0][0][2].full_name
    afas=AFASData.objects.filter(name=name).first()
    if afas:
        with transaction.atomic():
            entry[1].afas_data=afas
            entry[1].save()

In [ ]:
from PureOpenAlex.models import UTData, Department
from django.db.models import Q, Count, Window, F, Min, Max
from django.db.models.functions import RowNumber

duplicates = (
    UTData.objects.values("employee_id")
    .annotate(count=Count("employee_id"))
    .filter(count__gt=1)
)
for duplicate in duplicates:
    responses_to_check = UTData.objects.filter(
        employee_id=duplicate["employee_id"]
    ).annotate(
        row_number=Window(
            expression=RowNumber(),
            partition_by=[F("employee_id")],
            order_by=F("avatar").asc(),
        )
    )
    with transaction.atomic():
        responses_to_check.filter(row_number__gt=1).delete()
